# Joining Dataframes with SQL

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Spark SQL Query DataFrames').getOrCreate()

------

# read utilization csv file

In [16]:
data_path = '../Data'
file_path = data_path + '/utilization.csv'

In [19]:
df_util = spark.read.format('csv').options(header=False, inferSchema=True).load(file_path)

In [20]:
df_util.show(5)

+-------------------+---+----+----+---+
|                _c0|_c1| _c2| _c3|_c4|
+-------------------+---+----+----+---+
|03/05/2019 08:06:14|100|0.57|0.51| 47|
|03/05/2019 08:11:14|100|0.47|0.62| 43|
|03/05/2019 08:16:14|100|0.56|0.57| 62|
|03/05/2019 08:21:14|100|0.57|0.56| 50|
|03/05/2019 08:26:14|100|0.35|0.46| 43|
+-------------------+---+----+----+---+
only showing top 5 rows



In [21]:
# rename column
df_util = df_util.withColumnRenamed('_c0', 'event_datetime')\
                        .withColumnRenamed('_c1', 'server_id')\
                        .withColumnRenamed('_c2', 'cpu')\
                        .withColumnRenamed('_c3', 'free_memory')\
                        .withColumnRenamed('_c4', 'session_count')

In [26]:
df_util.show(5)

+-------------------+---------+----+-----------+-------------+
|     event_datetime|server_id| cpu|free_memory|session_count|
+-------------------+---------+----+-----------+-------------+
|03/05/2019 08:06:14|      100|0.57|       0.51|           47|
|03/05/2019 08:11:14|      100|0.47|       0.62|           43|
|03/05/2019 08:16:14|      100|0.56|       0.57|           62|
|03/05/2019 08:21:14|      100|0.57|       0.56|           50|
|03/05/2019 08:26:14|      100|0.35|       0.46|           43|
+-------------------+---------+----+-----------+-------------+
only showing top 5 rows



In [22]:
df_util.createOrReplaceTempView('vw_utilization')

------

# read in another server name file

In [23]:

file_path = data_path + '/server_name.csv'

df_server = spark.read.format('csv').options(header=True).load(file_path)

In [24]:
df_server.show(5)

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| 100 Server|
|      101| 101 Server|
|      102| 102 Server|
|      103| 103 Server|
|      104| 104 Server|
+---------+-----------+
only showing top 5 rows



In [27]:
df_server.createOrReplaceTempView('vw_server')

--------

# What are the unique server_id?

In [30]:
unique_server_id = spark.sql('SELECT DISTINCT(server_id) \
                              FROM vw_utilization \
                              ORDER BY server_id')

In [31]:
unique_server_id.show()

+---------+
|server_id|
+---------+
|      100|
|      101|
|      102|
|      103|
|      104|
|      105|
|      106|
|      107|
|      108|
|      109|
|      110|
|      111|
|      112|
|      113|
|      114|
|      115|
|      116|
|      117|
|      118|
|      119|
+---------+
only showing top 20 rows



In [32]:
unique_server_id.count()

50

# What are the min, max server_id ?

In [35]:
spark.sql('SELECT min(server_id), max(server_id) FROM vw_utilization').show()

+--------------+--------------+
|min(server_id)|max(server_id)|
+--------------+--------------+
|           100|           149|
+--------------+--------------+



# Joining two views

In [45]:
results = spark.sql('SELECT u.server_id, sn.server_name, u.session_count\
                FROM vw_utilization u \
                JOIN vw_server sn ON sn.server_id = u.server_id')

In [46]:
results.show()

+---------+-----------+-------------+
|server_id|server_name|session_count|
+---------+-----------+-------------+
|      100| 100 Server|           47|
|      100| 100 Server|           43|
|      100| 100 Server|           62|
|      100| 100 Server|           50|
|      100| 100 Server|           43|
|      100| 100 Server|           48|
|      100| 100 Server|           58|
|      100| 100 Server|           58|
|      100| 100 Server|           62|
|      100| 100 Server|           45|
|      100| 100 Server|           47|
|      100| 100 Server|           60|
|      100| 100 Server|           57|
|      100| 100 Server|           44|
|      100| 100 Server|           47|
|      100| 100 Server|           66|
|      100| 100 Server|           65|
|      100| 100 Server|           66|
|      100| 100 Server|           42|
|      100| 100 Server|           63|
+---------+-----------+-------------+
only showing top 20 rows



In [49]:
type(results)

pyspark.sql.dataframe.DataFrame

--------

# What are the names of those min, max server_id?

In [56]:
spark.sql('SELECT  sn.server_name, sn.server_id \
                FROM vw_server sn\
                WHERE sn.server_id = (SELECT min(server_id) FROM vw_utilization)').show()

+-----------+---------+
|server_name|server_id|
+-----------+---------+
| 100 Server|      100|
+-----------+---------+



In [57]:
spark.sql('SELECT  sn.server_name, sn.server_id \
                FROM vw_server sn\
                WHERE sn.server_id = (SELECT max(server_id) FROM vw_utilization)').show()

+-----------+---------+
|server_name|server_id|
+-----------+---------+
| 149 Server|      149|
+-----------+---------+



In [58]:
spark.sql('(SELECT  sn.server_name, sn.server_id \
                FROM vw_server sn\
                WHERE sn.server_id = (SELECT max(server_id) FROM vw_utilization))\
                UNION\
                (SELECT  sn.server_name, sn.server_id \
                FROM vw_server sn\
                WHERE sn.server_id = (SELECT min(server_id) FROM vw_utilization))').show()

+-----------+---------+
|server_name|server_id|
+-----------+---------+
| 100 Server|      100|
| 149 Server|      149|
+-----------+---------+

